# Root Finding and Optimization Methods

In computational physics and astrophysics, many problems reduce to two
fundamental kinds:
1. Root finding:
   Where does a function vanish?
   I.e., solve $f(x) = 0$.
2. Optimization:
   Where does a function reach an extremum (minimum or maximum)?
   I.e., solve $\nabla f(x) = 0$.

These two kind of problems are deeply connected.
Optimization often boils down to root finding on the derivative.
And root finding sometimes requires optimization-like strategies
to accelerate convergence.

Some classic examples of root finding include:
* Solving Kepler's equation $M = E - e \sin E$ to predict planetary
  orbits.
* Finding eigenfrequencies of stellar oscillations by locating roots
  of characteristic equations.

as well as optimization:
* Determining the launch angle of a projectile for maximum range.
* Fitting astrophysical models to observational data by minimizing a
  chi-square error function.
* Training machine learning models for data analysis in astronomy.

In simple cases, closed-form solutions exist (e.g. projectile motion
without air drag).
However, in realistic systems, equations are often nonlinear,
high-dimensional, and analytically unsolvable.
Numerical root finding and optimization methods are the only way to
solve these systems.

## General Framework of Interating Algorithms

Root finding means solving
\begin{align}
  f(x) = 0.
\end{align}
Many algorithms approach this through **iteration**:
starting from an initial guess, we repeatedly update $x$ until the
error is small.

### Fixed-Point Viewpoint

A powerful way to unify root-finding methods is to rewrite the problem
as a fixed-point equation:
\begin{align}
  x = g(x).
\end{align}

Then we can iterate:
\begin{align}
  x_{n+1} = g(x_n).
\end{align}

The solution $x^*$ is a *fixed point* of $g(x)$.
If the update rule is well chosen, the iteration converges to $x^*$.

### Convergence Criterion

Near the fixed point $x^*$, expand $g(x)$ in a Taylor series:
\begin{align}
  x_{n+1} = g(x_n) &\approx g(x^*) + g'(x^*) (x_n - x^*) = x^* + g'(x^*) (x_n - x^*).
\end{align}
Therefore,  
\begin{align}
  \frac{x_{n-1} - x^*}{x_n - x^*} &\approx g'(x^*).
\end{align}

It is clear that,
* If $|g'(x^*)| < 1$, the error shrinks, and the iteration converges.
* If $|g'(x^*)| > 1$, the iteration diverges.
* The closer $|g'(x^*)|$ is to 0, the faster the convergence.

This provides a general way to compare methods.

### Classical Root Finders

As we will soon see, classical root finding methods can be fitted into
this picture.
* Bisection Method:
  Is repeatedly shrinking an interval where the root must lie.
  The update rule is kind of a "double fixed-point scheme" where both
  the upper and lower bounds converge to the root.
  It is guaranteed to converge but only linearly.
* Newton–Raphson Method:
  Corresponds to choosing
  \begin{align}
    g(x) = x - \frac{f(x)}{f'(x)}.
  \end{align}
  If $f'(x^*) \neq 0$, this converges quadratically near the root.
* Secant Method:
  Uses the same Newton update rule, but replaces $f'(x)$ with a finite
  difference.
  This still fits into the fixed-point framework, with a convergence
  rate between bisection and Newton.

Thus, all root-finding methods can be viewed as different choices of
$g(x)$, with a trade-off between robustness and speed.

### Example

Let's solve $f(x)=x^2-2=0$.  
One possible choice of $g(x)$ is
\begin{align}
  g(x) = \tfrac12\left(x + \tfrac{2}{x}\right).
\end{align}
This is the
[ancient Babylonian update](https://www.sciencedirect.com/science/article/pii/S0315086022000477)
for $\sqrt{2}$.

In [ ]:
def g(x):
    return (x + 2/x)/2

x = 1.0
for i in range(5):
    print(f"Iteration {i}: x = {x}")
    x = g(x)

which converges very quickly to $\sqrt{2}$.

## Bisection Method

The Bisection Method is the simplest root-finding algorithm.
It trades speed for guaranteed convergence.
This makes it the "workhorse" method when robustness is more important
than efficiency.

Suppose $f(x)$ is continuous on an interval $[a,b]$.
If $f(a)$ and $f(b)$ have opposite signs, then by the
[Intermediate Value Theorem](https://en.wikipedia.org/wiki/Intermediate_value_theorem),
there exists at least one root in $(a,b)$.

The bisection method works by repeatedly halving the interval:
1. Compute the midpoint $m = (a+b)/2$.
2. Evaluate $f(m)$.
3. Select the half-interval $[a,m]$ or $[m,b]$ that contains the sign
   change.
4. Repeat until the interval is smaller than a desired tolerance.

### Convergence

Each step reduces the interval length by half:
\begin{align}
  (b-a) \to \tfrac12(b-a) \to \tfrac14(b-a) \to \cdots
\end{align}

After $n$ iterations, the uncertainty in the root is
\begin{align}
  \Delta x_n \approx \frac{b-a}{2^n}.
\end{align}

Although this convergence "exponentially" in terms of number of steps
$n$, we do not call this expoential convergence.
Instead, "convergence" in numerical analysis is usually from a step
size, i.e., $b-a$ for bisection method.
As $\Delta x_n$ scales only linear to $b-a$, bisection method is only
linear convergence.
It is reliable, but slower than other methods that we will introduce
later.

In [ ]:
def bisection(f, a, b, tol=1e-6, imax=100):
    if f(a)*f(b) >= 0:
        raise ValueError("f(a) and f(b) must have opposite signs.")
        
    for _ in range(imax):
        m = 0.5*(a+b)
        if f(m) == 0 or (b-a)/2 < tol:
            return m
        
        if f(a)*f(m) > 0:
            a = m
        else:
            b = m

    raise ValueError("Maximum iterations reached without convergence")

Let's solve $f(x)=x^3−x−2$,
which has a root between 1 and 2.

In [ ]:
def f(x):
    return x**3 - x - 2

root = bisection(f, 1, 2, tol=1e-6)
print("Approximate root:", root)
print("f(root) =", f(root))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

X = np.linspace(1, 2, 101)
Y = f(X)

plt.plot(X, Y, label="f(x)")
plt.plot(root, f(root), "o", label="Root")
plt.axhline(0, color="black", lw=1)
plt.xlabel("x")
plt.ylabel("f(x)")
plt.legend()

The bisection method is the most robust way to refine a root by
repeatedly shrinking the search interval.
Because it is so basic, it is one of the algorithm explicitly required
by 513!